In [6]:
import pandas as pd
import numpy as np
# import urllib2
# from bs4 import BeautifulSoup
# from imdbpie import Imdb
# from time import sleep
# import time
# import requests
import string
import csv
# import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
from sklearn.cross_validation import train_test_split

In [2]:
shows = pd.read_csv('shows_good_data.csv')

In [3]:
shows.head()

,first_year,fixed_runtime,is_action,is_adventure,is_animation,is_biography,is_comedy,is_crime,is_documentary,is_drama,...,keyword_supernatur,keyword_surreal,keyword_teenag,keyword_versu,keyword_villain,keyword_violenc,keyword_wife,keyword_woman,keyword_york,cancelled
0,2015,NaN,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015,30.0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,2015,21.0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2015,22.0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015,30.0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
def remove_punctuation(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

In [23]:
shows['stemmed_plot'] = shows['stemmed_plot'].dropna().apply(remove_punctuation)

In [24]:
y = shows['cancelled']
x = shows[[u'first_year', u'fixed_runtime',
                  u'is_action',         u'is_adventure',
               u'is_animation',         u'is_biography',
                  u'is_comedy',             u'is_crime',
             u'is_documentary',             u'is_drama',
                  u'is_family',           u'is_fantasy',
                    u'is_game',           u'is_history',
                  u'is_horror',             u'is_music',
                 u'is_musical',           u'is_mystery',
                    u'is_news',           u'is_reality',
                 u'is_romance',               u'is_sci',
                   u'is_short',             u'is_sport',
                    u'is_talk',          u'is_thriller',
                     u'is_war',           u'is_western',
              u'release_month',      u'release_weekday',
           u'release_monthday',         u'stemmed_plot',
           u'stemmed_keywords',        u'keyword_adult',
            u'keyword_african',        u'keyword_alien',
           u'keyword_american',        u'keyword_angel',
               u'keyword_anim',         u'keyword_base',
               u'keyword_best',        u'keyword_black',
               u'keyword_book',          u'keyword_boy',
          u'keyword_boyfriend',      u'keyword_brother',
         u'keyword_california',       u'keyword_celebr',
            u'keyword_charact',        u'keyword_child',
               u'keyword_citi',       u'keyword_comedi',
           u'keyword_comedian',        u'keyword_comic',
               u'keyword_cult',     u'keyword_daughter',
              u'keyword_death',       u'keyword_detect',
             u'keyword_doctor',         u'keyword_evil',
             u'keyword_famili',       u'keyword_father',
              u'keyword_femal',      u'keyword_fiction',
             u'keyword_friend',   u'keyword_friendship',
                u'keyword_gay',         u'keyword_girl',
         u'keyword_girlfriend',         u'keyword_hero',
              u'keyword_humor',      u'keyword_husband',
          u'keyword_interraci',    u'keyword_interview',
           u'keyword_investig',         u'keyword_joke',
               u'keyword_life',         u'keyword_live',
               u'keyword_love',         u'keyword_male',
                u'keyword_man',      u'keyword_marriag',
             u'keyword_mother',       u'keyword_murder',
             u'keyword_offic',
             u'keyword_parent',       u'keyword_parodi',
               u'keyword_play',        u'keyword_polic',
              u'keyword_power',  u'keyword_protagonist',
       u'keyword_relationship',        u'keyword_satir',
             u'keyword_school',       u'keyword_secret',
                u'keyword_sex',        u'keyword_share',
             u'keyword_sister',       u'keyword_sitcom',
             u'keyword_social',          u'keyword_son',
             u'keyword_spoken',        u'keyword_spoof',
              u'keyword_stand',      u'keyword_student',
          u'keyword_superhero',   u'keyword_supernatur',
            u'keyword_surreal',       u'keyword_teenag',
              u'keyword_versu',      u'keyword_villain',
            u'keyword_violenc',         u'keyword_wife',
              u'keyword_woman',         u'keyword_york']]

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x,y)

## TFIDF

In [10]:
## make lists of all plots before tfidf
## new df joining xtrain and ytrain
## subset of cancelled and renewed plots
## single list of words for cancelled plots, single list of words for renewed plots

In [26]:
training_set_for_plots = pd.concat([x_train, y_train], axis=1)

In [30]:
cancelled = training_set_for_plots[training_set_for_plots['cancelled'] == 1].dropna()
renewed = training_set_for_plots[training_set_for_plots['cancelled'] == 0].dropna()

In [39]:
cancelled_plots = list(cancelled['stemmed_plot'])
renewed_plots = list(renewed['stemmed_plot'])

In [42]:
new_cancelled_plots = ""
for i in cancelled_plots:
    new_cancelled_plots += (i + " ")

In [44]:
new_renewed_plots = ""
for i in renewed_plots:
    new_renewed_plots += (i + " ")

In [46]:
tvec = TfidfVectorizer(stop_words='english')
tvec.fit([new_cancelled_plots, new_renewed_plots])

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [50]:
tfidf_plotwords  = pd.DataFrame(tvec.transform([new_cancelled_plots, new_renewed_plots]).todense(),
                   columns=tvec.get_feature_names(),
                   index=['cancelled', 'renewed'])

In [51]:
tfidf_plotwords.head()

,10,100,10000,100000,1000000,101,1013,101st,10hour,10store,...,zoe,zoey,zombro,zoo,zookeep,zorak,zord,zordon,zowi,zuri
cancelled,0.002530,0.00000,0.000000,0.002530,0.000000,0.000000,0.000000,0.003555,0.003555,0.000000,...,0.010119,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
renewed,0.005856,0.00823,0.001646,0.001171,0.001646,0.001646,0.001646,0.000000,0.000000,0.001646,...,0.001171,0.006584,0.001646,0.003292,0.001646,0.001646,0.001646,0.001646,0.001646,0.001646


In [54]:
tfidf_plotwords.transpose().sort_values('cancelled', ascending=False).head(20).transpose()

,hi,ha,thi,life,live,new,famili,seri,work,friend,world,tri,wa,onli,brother,young,school,man,year,help
cancelled,0.612182,0.169488,0.146721,0.141662,0.141662,0.136602,0.134073,0.131543,0.126484,0.123954,0.116365,0.093598,0.091068,0.086009,0.083479,0.078420,0.075890,0.073361,0.070831,0.070831
renewed,0.553938,0.151074,0.151074,0.189721,0.195576,0.154587,0.161614,0.133507,0.106571,0.174496,0.097203,0.105400,0.073780,0.074951,0.053871,0.053871,0.101887,0.037476,0.066754,0.078465
